In [57]:
import mysql.connector
import csv
import os
import sys

from datetime import datetime
from collections import defaultdict

In [58]:
def getAnnotation(assembly_name, out_file):
    with open(out_file, 'w') as OUTPUT:
        cnx = mysql.connector.connect(
            user='genome', 
            host='genome-mysql.cse.ucsc.edu',
        )
        cursor = cnx.cursor(dictionary=True)

        query = ('SELECT '
                 + assembly_name + '.refGene.name,'
                 + assembly_name + '.refGene.chrom,'
                 + assembly_name + '.refGene.exonStarts,'
                 + assembly_name + '.refGene.exonEnds,'
                 + assembly_name + '.refGene.strand,'
                 + assembly_name + '.refGene.name2,'
                 + 'hgFixed.refLink.locusLinkId from '
                 + assembly_name + '.refGene '
                 + 'left outer join hgFixed.refLink on '
                 + assembly_name + '.refGene.name=hgFixed.refLink.mrnaAcc'
                )

        cursor.execute(query)
        
        transcripts = []
        for entry in cursor:
            transcripts.append(entry)
        sorted_transcripts = sorted(transcripts, key=lambda x: (x['chrom'],
                    int(x['exonStarts'].decode('UTF-8').split(',')[0]),
                    x['name'].split('_')[0],
                    x['name'].split('_')[1]))

        for entry in sorted_transcripts:
            starts = entry['exonStarts'].decode('UTF-8').split(',')[:-1]
            ends = entry['exonEnds'].decode('UTF-8').split(',')[:-1]
            
            for start, end in zip(starts, ends):
                OUTPUT.write('\t'.join([
                        entry['chrom'],
                        assembly_name + '.refGene',
                        'exon',
                        start,
                        end,
                        '.',
                        entry['strand'],
                        '.',
                        'gene_id "{}"; transcript_id "{}"'.format(
                            entry['name2'],
                            entry['name'],
                        ),
                    ]) + '\n')

In [59]:
# getAnnotation('dm3', 'temp.gtf')

In [60]:
annotations = [
    'hg19',
    'mm9',
    'hg38',
    'mm10',
    'dm6',
]

for annotation in annotations:
    getAnnotation(annotation, annotation + '.gtf')